# Cycling Network Analysis – Graz

Course: GIS Analysis Techniques 2  
Group: 1  
Authors: ...  


In [36]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
#import json
from shapely.geometry import Point 
from pathlib import Path
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import gc, time
import subprocess

print(sys.executable) #environment sanity check


c:\Users\kosch\anaconda3\envs\geo\python.exe


In [37]:
## Data ##
place = "Graz, Austria"
main_crs = "EPSG:31256"

In [38]:
## city and districts
gdf_boundaries = ox.features_from_place(
    place,
    tags={"boundary": "administrative"}
)

df_districts = gdf_boundaries[gdf_boundaries["admin_level"] == "9"]
df_districts = df_districts.to_crs(main_crs)

#df_districts.plot(edgecolor="black", figsize=(8, 8))
#plt.show()

In [39]:
## main University 
poi = [
    "Universitätsplatz 3, 8010 Graz",      
    "Rechbauerstraße 12, 8010 Graz",      
    "Neue Stiftingtalstraße 6, 8010 Graz",
    "Leonhardstraße 15, 8010 Graz",        
    "Körblergasse 126, 8010 Graz",        
    "Alte Poststraße 149, 8010 Graz",
    "Hasnerplatz 12, 8010 Graz",
    "Inffeldgasse 25, 8010 Graz"   
]

coords = [ox.geocode(addr) for addr in poi]
uni_points = [Point(lon, lat) for lat, lon in coords]

graz_unis = gpd.GeoDataFrame(
    {'name': poi},
    geometry=uni_points,
    crs="EPSG:4326"
).to_crs(main_crs)

#graz_unis.plot(figsize=(10,10), color="red", markersize=50, alpha=0.9)
#plt.show()

In [40]:
## Road network
G = ox.graph_from_place(
    place,
    network_type="bike",     
    simplify=True,
    retain_all=True           
)

G = ox.project_graph(G, to_crs=main_crs)

G_simple = nx.DiGraph(G)

#stats_json = ox.stats.basic_stats(G)
#print(json.dumps(stats_json, indent=2))

#fig, ax = ox.plot_graph(
#    G,
#    node_size=1,              
#    bgcolor="#040615",
#    edge_color="#d44e5c",
#    edge_linewidth=0.8
#)
#plt.show()

In [41]:
## Tram network
tram = ox.graph_from_place(
    place,
    custom_filter='["railway"="tram"]["service"!="yard"]["service"!="siding"]["service"!="spur"]',
    simplify=True,
    retain_all=True
)

tram = ox.project_graph(tram, to_crs=main_crs)

# to remove isolated edges
comps = list(nx.weakly_connected_components(tram)) 
tram_main = max(comps, key=len) 
tram_main = tram.subgraph(tram_main).copy()

#stats_json = ox.stats.basic_stats(tram_main)
#print(json.dumps(stats_json, indent=2))

#fig, ax = ox.plot_graph(
#    tram_main,
#    node_size=5,
#    bgcolor="#040615",
#    edge_color="#00d4ff",
#    edge_linewidth=1.2
#)

In [ ]:
# DEM
#path = Path(r"C:\Users\kosch\Desktop\final-project-group1-cycling-network\data_dem")
#FIX_NODATA = -9999.0

#raw_dirs = [path / s for s in ("352x20x","352x21x","353x20x","353x21x","354x20x","354x21x")]

#tiff = sorted({p for d in raw_dirs if d.exists() for p in d.rglob("*.tif")})

#city_geom = df_districts.to_crs(main_crs).dissolve().geometry.iloc[0]

# fixing
#fixed_dir = path / "_fixed_tiles"
#fixed_dir.mkdir(exist_ok=True)

#fixed_tifs = []
#for p in tiff:
#    out_p = fixed_dir / f"{p.parent.name}__{p.name}"
#
#    if not out_p.exists():
#        with rasterio.open(p) as src:
#            data = src.read(1).astype("float32")
#            data[data < -1e20] = FIX_NODATA

#            meta = src.meta.copy()
#            meta.update(dtype="float32", nodata=FIX_NODATA, count=1)

#        with rasterio.open(out_p, "w", **meta) as dst:
#            dst.write(data, 1)

#    fixed_tifs.append(out_p)

# mosaic
#srcs = [rasterio.open(p) for p in fixed_tifs]
#try:
#    mosaic, mosaic_transform = merge(srcs, nodata=FIX_NODATA, dtype="float32")
    meta = srcs[0].meta.copy()
#finally:
#    for s in srcs:
#        s.close()

#meta.update(
#    height=mosaic.shape[1],
#    width=mosaic.shape[2],
#    transform=mosaic_transform,
#    dtype="float32",
#    nodata=FIX_NODATA,
#    count=mosaic.shape[0],
#)

# crs
#dst_transform, dst_width, dst_height = calculate_default_transform(
#    meta["crs"], main_crs, meta["width"], meta["height"],
#    *rasterio.transform.array_bounds(meta["height"], meta["width"], meta["transform"])
#)

#reproj = np.empty((meta["count"], dst_height, dst_width), dtype="float32")
#for i in range(meta["count"]):
#    reproject(
#        source=mosaic[i],
#        destination=reproj[i],
#        src_transform=meta["transform"],
#        src_crs=meta["crs"],
#        dst_transform=dst_transform,
#        dst_crs=main_crs,
#        resampling=Resampling.bilinear,
#        src_nodata=FIX_NODATA,
#        dst_nodata=FIX_NODATA,
#    )

#reproj_meta = meta.copy()
#reproj_meta.update(crs=main_crs, transform=dst_transform, width=dst_width, height=dst_height)

# clip and save
#with rasterio.io.MemoryFile() as mf:
#    with mf.open(**reproj_meta) as ds:
#        ds.write(reproj)

#        out_img, out_transform = mask(ds, [city_geom], crop=True, nodata=FIX_NODATA)

#        out_meta = ds.meta.copy()
#        out_meta.update(height=out_img.shape[1], width=out_img.shape[2], transform=out_transform, nodata=FIX_NODATA)

#final_path = Path("..") / "data" / "processed" / "dem_graz.tif"
#with rasterio.open(final_path, "w", **out_meta) as dst:
#    dst.write(out_img)



In [43]:
## Preprocessing ##

In [44]:
## snapping
nodes, node_dists = ox.distance.nearest_nodes(
    G,
    X=graz_unis.geometry.x.to_numpy(),
    Y=graz_unis.geometry.y.to_numpy(),
    return_dist=True
)

graz_unis["nearest_node"] = nodes
graz_unis["dist_to_node_m"] = node_dists

edges, edge_dists = ox.distance.nearest_edges(
    G,
    X=graz_unis.geometry.x.to_numpy(),
    Y=graz_unis.geometry.y.to_numpy(),
    return_dist=True
)

graz_unis["nearest_edge"] = edges
graz_unis["dist_to_edge_m"] = edge_dists

graz_unis_snapped = graz_unis.drop_duplicates(subset=["nearest_node"]).copy()

#print(graz_unis[["name", "nearest_node", "dist_to_node_m"]])

In [ ]:
##slope
#dem_path = Path("..") / "data" / "processed" / "dem_graz.tif"
#slope_path = Path("..") / "data" / "processed" / "slope_graz.tif"

#result = subprocess.run([
#        "gdaldem", "slope", str(dem_path), str(slope_path),
#        "-s", "1",                 
#        "-compute_edges",             
#        "-of", "GTiff",               
#        "-co", "COMPRESS=LZW",        
#        "-co", "TILED=YES"           
#    ], 
#    capture_output=True, 
#    text=True, 
#    timeout=300
#    )